In [1]:
import pennylane as qml
import numpy as np

dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev, interface="autograd")
def circuit(x, y):
    # prepares the reference state |100>
    qml.BasisState(np.array([1,0,0]), wires=[0,1,2])
    # applies the single excitations
    qml.SingleExcitation(x, wires=[0,1])
    qml.SingleExcitation(y, wires=[0,2])
    return qml.state()

x = -2*np.arcsin(np.sqrt(1/3))
y = -2*np.arcsin(np.sqrt(1/2))
print(circuit(x, y))

[0.        +0.j 0.57735027+0.j 0.57735027+0.j 0.        +0.j
 0.57735027+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


In [2]:
tensor_state = circuit(x, y)
tensor_state

tensor([0.        +0.j, 0.57735027+0.j, 0.57735027+0.j, 0.        +0.j,
        0.57735027+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j], requires_grad=True)

In [3]:
tensor_state.reshape(2,2,2)

tensor([[[0.        +0.j, 0.57735027+0.j],
         [0.57735027+0.j, 0.        +0.j]],

        [[0.57735027+0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j]]], requires_grad=True)

Excitations on a fixed reference state and include only the excitations that preserve the spin orientation of the electron

In [5]:
n_particles = 3
n_qubits = 6

singles, doubles = qml.qchem.excitations(3, 6)
print(f"Single excitations = {singles}")
print(f"Double excitations = {doubles}")

Single excitations = [[0, 4], [1, 3], [1, 5], [2, 4]]
Double excitations = [[0, 1, 3, 4], [0, 1, 4, 5], [1, 2, 3, 4], [1, 2, 4, 5]]
